# Oasis Infobyte : Data Science Internship

# Step 1 : Importing Libraries

In [ ]:
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler

from sklearn.linear_model import LinearRegression
import statsmodels.api as sm
from sklearn.feature_selection import RFE
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.metrics import r2_score
from sklearn.metrics import mean_squared_error

# Step 2 : Importing Dataset

In [ ]:
a = pd.read_('car.csv')
a.head()

In [ ]:
a.tail()

In [ ]:
a.shape

In [ ]:
a.info()

In [ ]:
a.describe()

# Step 3 : Exploratory Data Analysis

In [ ]:
a.duplicated(subset = ['car_ID']).sum()

In [ ]:
a = a.drop(['car_ID'], axis =1)
a.isnull().sum()

In [ ]:
a['symboling'].value_counts()

In [ ]:
a['CarName'].value_counts()

In [ ]:
a['car_company'] = a['CarName'].apply(lambda x:x.split(' ')[0])
a.head()

In [ ]:
a = a.drop(['CarName'], axis =1)
a['car_company'].value_counts()

In [ ]:
a['car_company'].replace('toyouta', 'toyota',inplace=True)
a['car_company'].replace('Nissan', 'nissan',inplace=True)
a['car_company'].replace('maxda', 'mazda',inplace=True)
a['car_company'].replace('vokswagen', 'volkswagen',inplace=True)
a['car_company'].replace('vw', 'volkswagen',inplace=True)
a['car_company'].replace('porcshce', 'porsche',inplace=True)

In [ ]:
a['car_company'].value_counts()

In [ ]:
a['fueltype'].value_counts()

In [ ]:
a['aspiration'].value_counts()

In [ ]:
a['doornumber'].value_counts()

In [ ]:
def number_(x):
    return x.map({'four':4, 'two': 2})
    
a['doornumber'] = a[['doornumber']].apply(number_)


In [ ]:
a['doornumber'].value_counts()

In [ ]:
sns.distplot(a['wheelbase'],color='red')
plt.show()

In [ ]:
a['carlength'].value_counts().head()

In [ ]:
sns.distplot(a['carlength'],color='green')
plt.show()

In [ ]:
def convert_number(x):
    return x.map({'two':2, 'three':3, 'four':4,'five':5, 'six':6,'eight':8,'twelve':12})

a['cylindernumber'] = a[['cylindernumber']].apply(convert_number)

In [ ]:
a['cylindernumber'].value_counts()

In [ ]:
b =a.select_dtypes(include =['int64','float64'])
b.head()

In [ ]:
plt.figure(figsize = (30,30))
sns.pairplot(b)
plt.show()

In [ ]:
plt.figure(figsize = (20,20))
sns.heatmap(a.corr(), annot = True)
plt.show()

In [ ]:
categorical_cols = a.select_dtypes(include = ['object'])
categorical_cols.head()

In [ ]:
plt.figure(figsize = (20,12))
plt.subplot(3,3,1)
sns.boxplot(x = 'fueltype', y = 'price', data = a)
plt.subplot(3,3,2)
sns.boxplot(x = 'aspiration', y = 'price', data = a)
plt.subplot(3,3,3)
sns.boxplot(x = 'carbody', y = 'price', data = a)
plt.subplot(3,3,4)
sns.boxplot(x = 'drivewheel', y = 'price', data = a)
plt.subplot(3,3,5)
sns.boxplot(x = 'enginelocation', y = 'price', data = a)
plt.subplot(3,3,6)
sns.boxplot(x = 'enginetype', y = 'price', data = a)
plt.subplot(3,3,7)
sns.boxplot(x = 'fuelsystem', y = 'price', data = a)

In [ ]:
cars_dummies = pd.get_dummies(categorical_cols, drop_first = True)
cars_dummies.head()

In [ ]:
car_df  = pd.concat([a, cars_dummies], axis =1)
car_df = car_df.drop(['fueltype', 'aspiration', 'carbody', 'drivewheel', 'enginelocation',
       'enginetype', 'fuelsystem', 'car_company'], axis =1)

# Step 4 : Building ML model

In [ ]:
df_train, df_test = train_test_split(car_df, train_size = 0.7, test_size = 0.3, random_state = 100)

In [ ]:
df_train.shape

In [ ]:
df_test.shape

In [ ]:
b.columns

In [ ]:
col_list = ['symboling', 'doornumber', 'wheelbase', 'carlength', 'carwidth','carheight', 'curbweight', 'cylindernumber', 'enginesize', 'boreratio',
            'stroke', 'compressionratio', 'horsepower', 'peakrpm', 'citympg', 'highwaympg', 'price']
scaler = StandardScaler()
df_train[col_list] = scaler.fit_transform(df_train[col_list])

In [ ]:
df_train.describe()

In [ ]:
y_train = df_train.pop('price')
X_train = df_train

In [ ]:
lr = LinearRegression()
lr.fit(X_train,y_train)

In [ ]:
rfe = RFE(lr,step=15)
rfe.fit(X_train, y_train)

In [ ]:
list(zip(X_train.columns,rfe.support_,rfe.ranking_))

In [ ]:
cols = X_train.columns[rfe.support_]
cols

Model 1:

In [ ]:
X1 = X_train[cols]
X1_sm = sm.add_constant(X1)

lr_1 = sm.OLS(y_train,X1_sm).fit()
print(lr_1.summary())

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X1.columns
vif['VIF'] = [variance_inflation_factor(X1.values, i) for i in range(X1.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

In [ ]:
lr2 = LinearRegression()

rfe2 = RFE(lr2,step=10)
rfe2.fit(X_train,y_train)

In [ ]:
list(zip(X_train.columns,rfe2.support_,rfe2.ranking_))

In [ ]:
supported_cols = X_train.columns[rfe2.support_]
supported_cols 

Model 2:

In [ ]:
X2 = X_train[supported_cols]
X2_sm = sm.add_constant(X2)

model_2 = sm.OLS(y_train,X2_sm).fit()
print(model_2.summary())

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X2.columns
vif['VIF'] = [variance_inflation_factor(X2.values, i) for i in range(X2.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

Model 3:

In [ ]:
X3 = X2.drop(['car_company_subaru'], axis =1)
X3_sm = sm.add_constant(X3)

Model_3 = sm.OLS(y_train,X3_sm).fit()
print(Model_3.summary())

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X3.columns
vif['VIF'] = [variance_inflation_factor(X3.values, i) for i in range(X3.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

Model 4:

In [ ]:
X4 = X3.drop(['enginetype_ohcf'], axis =1)
X4_sm = sm.add_constant(X4)

Model_4 = sm.OLS(y_train,X4_sm).fit()
print(Model_4.summary())

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X4.columns
vif['VIF'] = [variance_inflation_factor(X4.values, i) for i in range(X4.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

Model 5:

In [ ]:
X5 = X4.drop(['car_company_peugeot'], axis =1)
X5_sm = sm.add_constant(X5)

Model_5 = sm.OLS(y_train,X5_sm).fit()
print(Model_5.summary())

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X5.columns
vif['VIF'] = [variance_inflation_factor(X5.values, i) for i in range(X5.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif


Model 6:

In [ ]:
X6 = X5.drop(['enginetype_l'], axis =1)
X6_sm = sm.add_constant(X6)

Model_6 = sm.OLS(y_train,X6_sm).fit()
print(Model_6.summary())

In [ ]:
vif = pd.DataFrame()
vif['Features'] = X6.columns
vif['VIF'] = [variance_inflation_factor(X6.values, i) for i in range(X6.shape[1])]
vif['VIF'] = round(vif['VIF'], 2)
vif = vif.sort_values(by = 'VIF', ascending = False)
vif

In [ ]:
y_train_pred = Model_6.predict(X6_sm)
y_train_pred.head()

In [ ]:
Residual = y_train- y_train_pred

In [ ]:
final_cols = X6.columns

In [ ]:
X_test_model6= X_test[final_cols]
X_test_model6.head()

In [ ]:
X_test_sm = sm.add_constant(X_test_model6)

In [ ]:
y_pred = Model_6.predict(X_test_sm)

In [ ]:
y_pred.head()

In [ ]:
c = [i for i in range(1,63,1)]
plt.plot(c, y_test,color = 'red')
plt.plot(c, y_pred,color = 'green')
plt.show()

In [ ]:
plt.scatter(y_test, y_pred)
plt.xlabel('y_test')
plt.ylabel('y_pred')
plt.show()

In [ ]:
r_squ = r2_score(y_test,y_pred)
r_squ